In [1]:
import os
import json
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.datasets import DatasetFolder, ImageFolder
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms, models
from torchsummary import summary
from tqdm import tqdm
from PIL import Image
import csv
import torch.cuda.amp as amp
import pandas as pd
import numpy as np
import cv2
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
import warnings

# warnings.filterwarnings("ignore")

torch.manual_seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
signs_dict = {
    "Speed limit (5km)": 0,
    "Speed limit (15km)": 1,
    "Speed limit (20km)": 2,
    "Speed limit (30km)": 3,
    "Speed limit (40km)": 4,
    "Speed limit (50km)": 5,
    "Speed limit (60km)": 6,
    "Speed limit (70km)": 7,
    "speed limit (80km)": 8,
    "speed limit (100km)": 9,
    "speed limit (120km)": 10,
    "End of speed limit": 11,
    "End of speed limit (50km)": 12,
    "End of speed limit (80km)": 13,
    "Dont overtake from Left": 14,
    "No stopping": 15,
    "No Uturn": 16,
    "No Car": 17,
    "No horn": 18,
    "No entry": 19,
    "No passage": 20,
    "Dont Go Right": 21,
    "Dont Go Left or Right": 22,
    "Dont Go Left": 23,
    "Dont Go straight": 24,
    "Dont Go straight or Right": 25,
    "Dont Go straight or left": 26,
    "Go right or straight": 27,
    "Go left or straight": 28,
    "Village": 29,
    "Uturn": 30,
    "ZigZag Curve": 31,
    "Bicycles crossing": 32,
    "Keep Right": 33,
    "Keep Left": 34,
    "Roundabout mandatory": 35,
    "Watch out for cars": 36,
    "Slow down and give way": 37,
    "Continuous detours": 38,
    "Slow walking": 39,
    "Horn": 40,
    "Uphill steep slope": 41,
    "Downhill steep slope": 42,
    "Under Construction": 43,
    "Heavy Vehicle Accidents": 44,
    "Parking inspection": 45,
    "Stop at intersection": 46,
    "Train Crossing": 47,
    "Fences": 48,
    "Dangerous curve to the right": 49,
    "Go Right": 50,
    "Go Left or right": 51,
    "Dangerous curve to the left": 52,
    "Go Left": 53,
    "Go straight": 54,
    "Go straight or right": 55,
    "Children crossing": 56,
    "Care bicycles crossing": 57,
    "Danger Ahead": 58,
    "Traffic signals": 59,
    "Zebra Crossing": 60,
    "Road Divider": 61
}

In [3]:
class SuperResolutionTransform:
    def __init__(self, model_path, scale=4):
        self.sr = cv2.dnn_superres.DnnSuperResImpl_create()
        self.sr.readModel(model_path)
        self.sr.setModel("espcn", scale)

    def __call__(self, img):
        img_cv = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
        upsampled_img_cv = self.sr.upsample(img_cv)
        upsampled_img = Image.fromarray(cv2.cvtColor(upsampled_img_cv, cv2.COLOR_BGR2RGB))
        return upsampled_img

transform = transforms.Compose([
    # transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5)), # 高斯模糊
    # SuperResolutionTransform("ESPCN_x4.pb"),  # 超分辨率重构
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),  # 颜色调整
    transforms.Grayscale(num_output_channels=3),
    # transforms.Resize((50,50)),
    transforms.Resize((224,224)),
    # transforms.RandomHorizontalFlip(), # 随机翻转
    transforms.RandomRotation(degrees=15),  # 随机旋转
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [4]:
# 自定义数据集类
class CustomImageDataset(Dataset):
    def __init__(self, root_dir, transform=None, label_dict=signs_dict):
        self.root_dir = root_dir
        self.transform = transform
        self.label_dict = label_dict
        self.image_paths = []
        self.labels = []

        for label_name, label_idx in label_dict.items():
            label_dir = os.path.join(root_dir, label_name)
            for img_name in os.listdir(label_dir):
                img_path = os.path.join(label_dir, img_name)
                self.image_paths.append(img_path)
                self.labels.append(label_idx)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]
        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return image, label

In [5]:
# 引入数据集
dataset = CustomImageDataset(root_dir='./data/train_set/', transform=transform)

# 定义训练集和验证集的比例
train_ratio = 0.90  # 90%的数据用于训练
val_ratio = 1 - train_ratio  # 剩下的用于验证

# 计算划分后的样本数量
train_size = int(train_ratio * len(dataset))
val_size = len(dataset) - train_size

# 划分数据集
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# 创建训练集和验证集的数据加载器
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

In [6]:
class CA_Block(nn.Module):
    def __init__(self, channel, h, w, reduction=16):
        super(CA_Block, self).__init__()
 
        self.h = h
        self.w = w
 
        self.avg_pool_x = nn.AdaptiveAvgPool2d((h, 1))
        self.avg_pool_y = nn.AdaptiveAvgPool2d((1, w))
 
        self.conv_1x1 = nn.Conv2d(in_channels=channel, out_channels=channel//reduction, kernel_size=1, stride=1, bias=False)
 
        self.relu = nn.ReLU()
        self.bn = nn.BatchNorm2d(channel//reduction)
 
        self.F_h = nn.Conv2d(in_channels=channel//reduction, out_channels=channel, kernel_size=1, stride=1, bias=False)
        self.F_w = nn.Conv2d(in_channels=channel//reduction, out_channels=channel, kernel_size=1, stride=1, bias=False)
 
        self.sigmoid_h = nn.Sigmoid()
        self.sigmoid_w = nn.Sigmoid()
 
    def forward(self, x):
 
        x_h = self.avg_pool_x(x).permute(0, 1, 3, 2)
        x_w = self.avg_pool_y(x)
 
        x_cat_conv_relu = self.relu(self.conv_1x1(torch.cat((x_h, x_w), 3)))
 
        x_cat_conv_split_h, x_cat_conv_split_w = x_cat_conv_relu.split([self.h, self.w], 3)
 
        s_h = self.sigmoid_h(self.F_h(x_cat_conv_split_h.permute(0, 1, 3, 2)))
        s_w = self.sigmoid_w(self.F_w(x_cat_conv_split_w))
 
        out = x * s_h.expand_as(x) * s_w.expand_as(x)
 
        return out

In [7]:
class ResnetCA(nn.Module):
    def __init__(self, model=models.resnet18(pretrained=False)):
        super(ResnetCA,self).__init__()
        self.resnet = nn.Sequential(*list(model.children())[:-2])
        self.ca = CA_Block(channel=512, h=7, w=7)
        self.avg_pool = nn.Sequential(*list(model.children())[-2:])[0]
        self.fc = nn.Linear(in_features=512, out_features=62, bias=True)
    def forward(self, x):
        x=self.resnet(x)
        # x=x.view(-1,49,512)
        x=self.ca(x)
        x=x.view(-1,512,7,7)
        x=self.avg_pool(x)
        x=x.view(x.size(0), -1)
        x=self.fc(x)
        return x

resnet_ca = ResnetCA()

E:\envs\torch\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
E:\envs\torch\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [8]:
summary(resnet_ca,input_size=(3,224,224),device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]          36,864
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
       BasicBlock-11           [-1, 64, 56, 56]               0
           Conv2d-12           [-1, 64, 56, 56]          36,864
      BatchNorm2d-13           [-1, 64, 56, 56]             128
             ReLU-14           [-1, 64,

In [9]:
def evaluate(model, data_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    all_predictions = []
    all_labels = []

    with torch.no_grad():
        for images, labels in data_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            all_predictions.extend(predicted.cpu().tolist())
            all_labels.extend(labels.cpu().tolist())
    
    loss = running_loss / len(data_loader.dataset)
    acc = correct / total
    f1 = f1_score(all_labels, all_predictions, average='macro')
    report = classification_report(all_labels, all_predictions,zero_division=0)
    
    return loss, acc, f1, report

def train(model, train_loader, val_loader, criterion, optimizer, device, epochs=10, logs='./logs/training_log.csv', save_model=False):
    os.makedirs(os.path.dirname(logs), exist_ok=True)
    classification_report_last = './logs/classification_report_last.txt'
    classification_report_best = './logs/classification_report_best.txt'
    
    with open(logs, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Epoch', 'Train Loss', 'Train Acc', 'Train F1', 'Val Loss', 'Val Acc', 'Val F1'])

    best_val_f1 = 0.0
    best_val_acc = 0.0
    total_steps = epochs
    progress_bar = tqdm(total=total_steps, desc="epochs")

    scaler = amp.GradScaler()  # 创建混合精度训练的缩放器

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        all_train_predictions = []
        all_train_labels = []
        
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            
            with amp.autocast():  # 开始混合精度计算
                outputs = model(images)
                loss = criterion(outputs, labels)
            
            scaler.scale(loss).backward()  # 缩放损失值以进行反向传播
            
            scaler.step(optimizer)  # 调整模型参数
            scaler.update()  # 更新缩放器
            
            running_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            all_train_predictions.extend(predicted.cpu().tolist())
            all_train_labels.extend(labels.cpu().tolist())
        
        train_loss = running_loss / len(train_loader.dataset)
        train_acc = correct / total
        train_f1 = f1_score(all_train_labels, all_train_predictions, average='macro')
        
        val_loss, val_acc, val_f1, val_report = evaluate(model, val_loader, criterion, device)
        
        with open(logs, mode='a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([epoch + 1, train_loss, train_acc, train_f1, val_loss, val_acc, val_f1])
        
        # 保存最终模型的分类报告
        with open(classification_report_last, 'w') as f:
            f.write(val_report)
        
        # 保存最佳模型的分类报告
        if val_acc >= best_val_acc:
            best_val_acc = val_acc
            best_val_f1 = val_f1
            best_val_report = val_report
            torch.save(model, 'model/best_model.pth')
            
            with open(classification_report_best, 'w') as f:
                f.write(best_val_report)

        progress_bar.update(1)
    
    progress_bar.close()

In [10]:
# 定义损失函数、优化器和设备
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet_ca.parameters(), lr=0.001)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [11]:
# 训练模型
train(resnet_ca.to(device), train_loader, val_loader, criterion, optimizer, device, epochs=100,save_model=True)

epochs: 100%|██████████████████████████████████████████████████████████████████████| 100/100 [1:58:35<00:00, 71.15s/it]


In [12]:
# 定义数据集类
class ImageDataset(Dataset):
    def __init__(self, folder_path, transform=None):
        self.folder_path = folder_path
        self.image_files = os.listdir(folder_path)
        self.transform = transform

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = os.path.join(self.folder_path, self.image_files[idx])
        image = Image.open(img_name).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, self.image_files[idx]   

# 定义预处理变换
transform = transforms.Compose([
    SuperResolutionTransform("ESPCN_x4.pb"),  # 超分辨率重构
    transforms.Grayscale(num_output_channels=3),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),  # 颜色调整
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
    
# 加载模型
model_path = "./model/best_model.pth"
model = torch.load(model_path)
model.eval()
model.to(device)

# 加载数据集
test_dataset = ImageDataset("./data/test_set/unknow/", transform=transform)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

# 预测并保存结果到CSV
results = []
for images, filenames in test_loader:
    with torch.no_grad():
        outputs = model(images.to(device))
        _, predicted = torch.max(outputs, 1)
        results.append((filenames[0], predicted.item()))

# 将结果保存到CSV
df = pd.DataFrame(results, columns=["ImageID", "label"])
df.to_csv("predictions.csv", index=False)